In [ ]:
import os
import cv2
import zipfile
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image

import warnings

warnings.filterwarnings("ignore")

#### config file

In [1]:
RAW_DATA_PATH = "../../data/raw/"
PROCESSED_DATA_PATH = "../../data/processed/"
FILES_PATH = "../../research/files/"
TRAIN_MODELS = "../../checkpoints/train_models/"
BEST_MODELS = "../../checkpoints/best_models/"
BEST_MODEL = "../../checkpoints/best_model/"
TRAIN_IMAGES = "../../outputs/train_images/"
TEST_IMAGES = "../../outputs/test_images/"
GIF_FILE = "../../outputs/train_gif"
METRICS_PATH = "../../metrics/"
MODEL_HISTORY = "../../checkpoints/model_history/"

#### Utils file

In [ ]:
def dump(value, filename):
    if (value is not None) and (filename is not None):
        joblib.dump(value=value, filename=filename)
    else:
        raise Exception("Value or filename not provided")
    
def load(filename):
    if (filename is not None):
        return joblib.load(filename=filename)
    else:
        raise Exception("Filename not provided")
    
def weight_init(m, he_normal = False):
    classname = m.__class__.__name__
    
    if classname.find("Conv") != -1:
        if he_normal:
            nn.init.kaiming_normal_(
                m.weight, mode="fan_out", nonlinearity="relu")
        else:
            nn.init.normal_(m.weight, mean=0.0, std=0.02)
            
    elif classname.find("BatchNorm")!= -1:
        if he_normal:
            nn.init.kaiming_normal_(
                m.weight, mode="fan_out", nonlinearity="relu")        
        else:
            nn.init.normal_(m.weight, mean=1.0, std=0.02)
            nn.init.constant_(m.bias, 0.0)
            
            
def device_init(device = "mps"):
    if device == "mps":
        return torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    elif device == "cuda":
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        return torch.device("cpu")

#### Dataloader

In [ ]:
class Loader(Dataset):
    def __init__(self, image_path = None, in_channels = 3, batch_size = 1, image_size = 64):
        self.image_path = image_path
        self.batch_size = batch_size
        self.in_channels = in_channels
        self.image_size = image_size

        self.train_images = list()
        self.train_labels = list()
        self.test_images = list()
        self.test_labels = list()

    def image_normalized(self, lr_images=True):
        return transforms.Compose(
            [
                (
                    transforms.Resize((self.image_size, self.image_size))
                    if lr_images
                    else transforms.Resize((self.image_size*4, self.image_size*4))
                ),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
            ]
        )

    def unzip_folder(self):
        if os.path.exists(RAW_DATA_PATH):
            with zipfile.ZipFile(self.image_path, "r") as zip_file:
                zip_file.extractall(os.path.join(RAW_DATA_PATH, "images"))
        else:
            raise Exception("Raw data folder not found".capitalize())

    def extract_images(self):
        if os.path.exists(RAW_DATA_PATH):
            image_path = os.path.join(RAW_DATA_PATH, "images")

            train_images = os.path.join(image_path, os.listdir(image_path)[0])
            test_images = os.path.join(image_path, os.listdir(image_path)[1])

            for idx, path in enumerate([train_images, test_images]):
                categories = os.listdir(path)

                for category in categories:
                    images = os.path.join(path, category)

                    for image in os.listdir(images):
                        self.image_path = os.path.join(images, image)
                        self.image = cv2.imread(self.image_path)

                        if self.image is not None:
                            if idx == 0:
                                self.train_images.append(
                                    self.image_normalized(lr_images=True)(
                                        Image.fromarray(self.image)
                                    )
                                )

                                self.train_labels.append(
                                    self.image_normalized(lr_images=False)(
                                        Image.fromarray(self.image)
                                    )
                                )
                            else:
                                self.test_images.append(
                                    self.image_normalized(lr_images=True)(
                                        Image.fromarray(self.image)
                                    )
                                )

                                self.test_labels.append(
                                    self.image_normalized(lr_images=False)(
                                        Image.fromarray(self.image)
                                    )
                                )
                        else:
                            continue
            return {
                "train_images": self.train_images,
                "train_labels": self.train_labels,
                "test_images": self.test_images,
                "test_labels": self.test_labels
                }
        else:
            raise Exception("Raw data folder not found".capitalize())

    def create_dataloader(self):
        try:
            images = self.extract_images()
        except Exception as e:
            print("Error in extracting images".capitalize())
        else:
            train_dataloader = DataLoader(
                dataset=list(zip(images["train_images"], images["train_labels"])), batch_size=self.batch_size, shuffle=True)
            test_dataloader = DataLoader(
                dataset=list(zip(images["test_images"], images["test_labels"])), batch_size=self.batch_size*64, shuffle=True)

            if os.path.join(PROCESSED_DATA_PATH):
                try:
                    dump(
                        value=train_dataloader, filename=os.path.join(PROCESSED_DATA_PATH, "train_dataloader.pkl"))
                except Exception as e:
                    print("Error in dumping train dataloader".capitalize())

                try:   
                    dump(
                        value=test_dataloader, filename=os.path.join(PROCESSED_DATA_PATH, "test_dataloader.pkl"))
                except Exception as e:
                    print("Error in dumping test dataloader".capitalize())
            else:
                raise Exception("Processed data folder not found".capitalize())

    @staticmethod
    def display_images():
        if os.path.exists(PROCESSED_DATA_PATH):
            images = list()
            labels = list()

            train_images = load(filename=os.path.join(PROCESSED_DATA_PATH, "train_dataloader.pkl"))

            for index, (image, label) in enumerate(train_images):

                if index !=64:
                    images.append(image)
                    labels.append(label)
                else:
                    break

            plt.figure(figsize=(40, 25))

            for index, (lr_image, hr_image) in enumerate(zip(images, labels)):

                lr_image = lr_image.squeeze().permute(1, 2, 0)
                lr_image = lr_image.cpu().detach().numpy()
                lr_image = (lr_image - lr_image.min())/(lr_image.max() - lr_image.min())

                plt.subplot(2 * 8, 2 * 8, 2 * index + 1)
                plt.imshow(lr_image, cmap="gray")
                plt.title("lr_image".lower())
                plt.axis("off")

                hr_image = hr_image.squeeze().permute(1, 2, 0)
                hr_image = hr_image.cpu().detach().numpy()
                hr_image = (hr_image - hr_image.min())/(hr_image.max() - hr_image.min())

                plt.subplot(2 * 8, 2 * 8, 2 * index + 2)
                plt.imshow(hr_image, cmap="gray")
                plt.title("hr_image".lower())
                plt.axis("off")

            plt.tight_layout()
            plt.show()

    @staticmethod
    def details_dataset():
        if os.path.exists(PROCESSED_DATA_PATH):
            train_dataloader = load(filename=os.path.join(PROCESSED_DATA_PATH, "train_dataloader.pkl"))
            test_dataloader = load(filename=os.path.join(PROCESSED_DATA_PATH, "test_dataloader.pkl"))

            train = sum(data.size(0) for data, _ in train_dataloader)
            test = sum(data.size(0) for data, _ in test_dataloader)

            train_lr_images, train_hr_images = next(iter(train_dataloader))
            test_lr_images, test_hr_images = next(iter(test_dataloader))

            print("Train images: {}".format(train))
            print("Test images: {}".format(test))

            print(
                "Train: lower resolution images shape: {}".format(
                    train_lr_images.squeeze().size()
                ).capitalize()
            )
            print(
                "Train: higher resolution images shape: {}".format(
                    train_hr_images.squeeze().size()
                ).capitalize()
            )
            print(
                "Test: lower resolution images shape: {}".format(
                    test_lr_images.squeeze().size()
                ).capitalize()
            )
            print(
                "Test: higher resolution images shape: {}".format(
                    test_hr_images.squeeze().size()
                ).capitalize()
            )

        else:
            raise Exception("Processed data folder not found".capitalize())
        
        
        
if __name__ == "__main__":
    loader = Loader(image_path="/Users/shahmuhammadraditrahman/Desktop/images.zip")
    loader.unzip_folder()
    loader.create_dataloader()
    
    loader.details_dataset()
    loader.display_images()

#### Generator

In [ ]:
class InputBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(InputBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.kernel_size = 9
        self.stride = 1
        self.padding = 4
        
        try:
            self.model = self.input_block()
        except Exception as e:
            print("Input block not implemented")
        
    def input_block(self):
        layers = OrderedDict()
        
        layers["conv"] = nn.Conv2d(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            kernel_size = self.kernel_size,
            stride = self.stride,
            padding = self.padding
        )
        layers["PReLU"] = nn.PReLU()
        
        return nn.Sequential(layers)
    
    def forward(self, x=None):
        if x is not None:
            return self.model(x)
        else:
            raise Exception("Input block not implemented")

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None, index = None):
        super(ResidualBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.index = index
        
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        
        try:
            self.model = self.residual_block()
        except Exception as e:
            print("Residual block not implemented")
        
    def residual_block(self):
        layers = OrderedDict()
        
        layers["conv{}".format(self.index)] = nn.Conv2d(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding
        )
        layers["batchnorm{}".format(self.index)] = nn.BatchNorm2d(
            num_features=self.out_channels)
        
        layers["PReLU{}".format(self.index)] = nn.PReLU()
        layers["conv{}".format(self.index+1)] = nn.Conv2d(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding
        )
        layers["batchnorm{}".format(self.index+1)] = nn.BatchNorm2d(
            num_features=self.out_channels)
        
        return nn.Sequential(layers)
    
    def forward(self, x=None):
        if x is not None:
            return x + self.model(x)
        else:
            raise Exception("Residual block not implemented")

In [ ]:
class MiddleBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(MiddleBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        
        try:
            self.model = self.middle_block()
        except Exception as _:
            print("Middle block not implemented")
            
    def middle_block(self):
        layers = OrderedDict()
        
        layers["conv"] = nn.Conv2d(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            kernel_size = self.kernel_size,
            stride = self.stride,
            padding = self.padding
        )
        
        layers["batchnorm"] = nn.BatchNorm2d(num_features=self.out_channels)
        
        return nn.Sequential(layers)
    
    def forward(self, x = None, skip_info = None):
        if (x is not None) and (skip_info is not None):
            return self.model(x) + skip_info 
        else:
            raise Exception("Middle block not implemented".capitalize())     

In [ ]:
class UpSampleBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None, is_first_block = False, index = None):
        super(UpSampleBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.is_first_block = is_first_block
        self.index = index
        
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.factor = 2
        
        try:
            self.model = self.up_sample_block()
        except Exception as _:
            print("Up sample block not implemented".capitalize())
            
    def up_sample_block(self):
        layers = OrderedDict()
        layers["conv{}".format(self.index)] = nn.Conv2d(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding
        )
        layers["pixel_shuffle{}".format(self.index)] = nn.PixelShuffle(
            upscale_factor=self.factor)
        
        if self.is_first_block:
            layers["PReLU"] = nn.PReLU()
            
        return nn.Sequential(layers)
        
    def forward(self, x):
        if x is not None:
            return self.model(x)
        else:
            raise Exception("Up sample block not implemented".capitalize())

In [ ]:
class OutputBlock(nn.Module):
    def __init__(self, in_channels = None, out_channels = None):
        super(OutputBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.kernel_size = 9
        self.stride = 1
        self.padding = 4
        
        try:
            self.model = self.output_block()
        except Exception as _:
            print("Output block not implemented".capitalize())
            
    def output_block(self):
        layers = OrderedDict()
        
        layers["conv"] = nn.Conv2d(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            kernel_size = self.kernel_size,
            stride=self.stride,
            padding = self.padding
        )
        layers["tanh"] = nn.Tanh()
        
        return nn.Sequential(layers)
    
    def forward(self, x):
        if x is not None:
            return self.model(x)
        else:
            raise Exception("Output block not implemented".capitalize())

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.num_repetitive = 16
        
        self.input_block = InputBlock(in_channels = 3, out_channels = 64)
        
        self.residual_block = nn.Sequential(
            *[ResidualBlock(in_channels=64, out_channels=64, index=index) for index in range(self.num_repetitive)])
        
        self.middle_block = MiddleBlock(in_channels=64, out_channels=64)
        
        self.up_sample = nn.Sequential(
            *[UpSampleBlock(in_channels=64, out_channels=256, is_first_block=is_first_block, index=index)
            for index, is_first_block in enumerate([True, False])])
        
        self.out_block = OutputBlock(in_channels=64, out_channels=3)
        
    def forward(self, x):
        if x is not None:
            input = self.input_block(x)
            residual = self.residual_block(input)
            middle = self.middle_block(residual, input)
            upsample = self.up_sample(middle)
            output = self.out_block(upsample)
            
            return output
        
        else:
            raise Exception("Generator not implemented".capitalize())

#### Discriminator

In [ ]:
class InputBlock(nn.Module):
    def __init__(self, in_channels=None, out_channels=None):
        super(InputBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.kernel_size = 3
        self.stride = 1
        self.padding = 1

        self.model = self.input_block()

    def input_block(self):
        return nn.Sequential(
            nn.Conv2d(
                self.in_channels,
                self.out_channels,
                self.kernel_size,
                self.stride,
                self.padding,
            ),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(
                self.out_channels,
                self.out_channels,
                self.kernel_size,
                self.stride,
                self.padding,
            ),
            nn.BatchNorm2d(num_features=self.out_channels),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
        )

    def forward(self, x):
        if x is not None:
            return self.model(x)

In [ ]:
class FeatureBlock(nn.Module):
    def __init__(
        self, in_channels=None, out_channels=None, kernel_size=3, stride=2, padding=1
    ):
        super(FeatureBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.model = self.feature_block()

    def feature_block(self):
        return nn.Sequential(
            nn.Conv2d(
                self.in_channels,
                self.out_channels,
                self.kernel_size,
                self.stride,
                self.padding,
            ),
            nn.BatchNorm2d(num_features=self.out_channels),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
        )

    def forward(self, x):
        if x is not None:
            return self.model(x)

In [ ]:
class OutBlock(nn.Module):
    def __init__(self, in_channels=None, out_channels=None):
        super(OutBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = out_channels

        self.model = self.output_block()

    def output_block(self):
        return nn.Sequential(
            nn.Conv2d(self.in_channels * 8, self.in_channels * 16, self.kernel_size),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(self.in_channels * 16, 1, self.kernel_size),
            nn.Tanh(),
        )

    def forward(self, x):
        if x is not None:
            return self.model(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=None, out_channels=None):
        super(Discriminator, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.filters = out_channels

        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.num_repetitive = 7
        self.layers = []

        self.input = InputBlock(
            in_channels=self.in_channels, out_channels=self.out_channels
        )

        for index in range(self.num_repetitive):
            if index % 2:
                self.layers.append(
                    FeatureBlock(
                        in_channels=self.out_channels,
                        out_channels=self.out_channels * 2,
                    )
                )

                self.out_channels = self.out_channels * 2

            else:
                self.layers.append(
                    FeatureBlock(
                        in_channels=self.out_channels, out_channels=self.out_channels
                    )
                )

                self.out_channels = self.out_channels

        self.features = nn.Sequential(*self.layers)

        self.avg_pool = nn.AdaptiveMaxPool2d(output_size=1)

        self.output = OutBlock(in_channels=self.filters, out_channels=1)

    def forward(self, x):
        input = self.input(x)
        features = self.features(input)
        output = self.output(self.avg_pool(features))

        return output.view(-1)

In [ ]:
netD = Discriminator(in_channels=3, out_channels=64)

netD(torch.randn(1, 3, 256, 256)).size()

#### Feature Extractor - VGG16

In [ ]:
class VGG16(nn.Module):
    def __init__(self, pretrained = True, is_vgg16 = False):
        super(VGG16, self).__init__()
        
        self.pretrained = pretrained
        self.is_vgg16 = is_vgg16
        self.num_layers = 18
        
        if self.is_vgg16:
            self.model = models.vgg16(pretrained=self.pretrained)
            
            for params in self.model.parameters():
                params.requires_grad = False
                
        else:
            self.model = models.vgg19(pretrained=self.pretrained)
            
            for params in self.model.parameters():
                params.requires_grad = False
                
        self.model = nn.Sequential(*list(self.model.features.children())[:self.num_layers])
        
    
    def forward(self, x):
        if x is not None:
            return self.model(x)
        else:
            raise Exception("Feature Extractor not implemented".capitalize())
        

if __name__ == "__main__":
    feature_extractor = VGG16(pretrained=True, is_vgg16=False)
    
    images = torch.randn(1, 3, 256, 256)
    
    print(feature_extractor(images).size())

#### Helpers method

In [ ]:
def load_dataloader():
    if os.path.exists(PROCESSED_DATA_PATH):
        train_dataloader = load(
            filename=os.path.join(PROCESSED_DATA_PATH, "train_dataloader.pkl")
        )
        test_dataloader = load(
            filename=os.path.join(PROCESSED_DATA_PATH, "test_dataloader.pkl")
        )
    else:
        raise Exception("Dataset not found".capitalize())

    return train_dataloader, test_dataloader


def helpers(**kwargs):

    lr = kwargs["lr"]
    beta1 = kwargs["beta1"]
    adam = kwargs["adam"]
    SGD = kwargs["SGD"]
    device = kwargs["device"]
    lr_scheduler = kwargs["is_lr_scheduler"]
    netG = netD = optimizerG = optimizerD = schedulerG = schedulerD = None

    try:
        train_dataloader, test_dataloader = load_dataloader()

    except Exception as e:
        print("The exception is: ", e)

    if adam:
        try:
            netG = Generator().to(device)
            netD = Discriminator(in_channels=3, out_channels=64).to(device)

        except Exception as e:
            print("The exception caught in the section # {}".format(e).capitalize())
        else:

            optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))
            optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))

    elif SGD:
        try:
            netG = Generator().to(device)
            netD = Discriminator(in_channels=3, out_channels=64).to(device)

        except Exception as e:
            print("The exception caught in the section # {}".format(e).capitalize())
        else:
            optimizerG = optim.SGD(netG.parameters(), lr=lr, momentum=beta1)
            optimizerD = optim.SGD(netD.parameters(), lr=lr, momentum=beta1)

    if lr_scheduler:
        schedulerG = StepLR(
            optimizerG,
            step_size=10,
            gamma=0.1,
        )
        schedulerD = StepLR(
            optimizerD,
            step_size=10,
            gamma=0.1,
        )

    try:
        content_loss = VGG16(pretrained=True).to(device)
        adversarial_loss = nn.MSELoss()

    except Exception as e:
        print("The exception caught in the section # {}".format(e).capitalize())

    return {
        "train_dataloader": train_dataloader,
        "test_dataloader": test_dataloader,
        "netG": netG,
        "netD": netD,
        "optimizerG": optimizerG,
        "optimizerD": optimizerD,
        "schedulerG": schedulerG,
        "schedulerD": schedulerD,
        "adversarial_loss": adversarial_loss,
        "content_loss": content_loss,
    }

#### Training

In [2]:
class Trainer:
    def __init__(
        self,
        epochs=100,
        lr=0.0002,
        content_loss=1e-2,
        device="mps",
        adam=True,
        SGD=False,
        beta1=0.5,
        is_l1=False,
        is_l2=False,
        is_elastic_net=False,
        is_lr_scheduler=False,
        is_weight_init=False,
        is_weight_clip=False,
        display=True,
    ):
        self.epochs = epochs
        self.lr = lr
        self.content_loss = content_loss
        self.device = device
        self.adam = adam
        self.SGD = SGD
        self.beta1 = beta1
        self.is_l1 = is_l1
        self.is_l2 = is_l2
        self.is_elastic_net = is_elastic_net
        self.is_lr_scheduler = is_lr_scheduler
        self.is_weight_init = is_weight_init
        self.is_weight_clip = is_weight_clip
        self.is_display = display

        try:
            init = helpers(
                lr=self.lr,
                beta1=self.beta1,
                adam=self.adam,
                SGD=self.SGD,
                device=self.device,
                is_lr_scheduler=self.is_lr_scheduler,
            )
        except Exception as e:
            print("The exception caught in the section # {}".format(e).capitalize())

        else:
            self.train_dataloader = init["train_dataloader"]
            self.test_dataloader = init["test_dataloader"]

            if self.is_weight_init:
                self.netG = init["netG"].apply(weight_init)
                self.netD = init["netD"].apply(weight_init)

            else:
                self.netG = init["netG"]
                self.netD = init["netD"]

            self.optimizerG = init["optimizerG"]
            self.optimizerD = init["optimizerD"]
            self.schedulerG = init["schedulerG"]
            self.schedulerD = init["schedulerD"]
            self.adversarial_loss = init["adversarial_loss"]
            self.criterion_content = init["criterion_loss"]

            self.infinity = float("inf")
            self.clip_value = 0.01
            self.loss_track = {"netG": list(), "netD": list()}
            self.history = {"netG": list(), "netD": list()}

    def l1(self, model):
        if model is not None:
            return (
                torch.norm(input=params, p=1) for params in model.parameters()
            ).sum()
        else:
            raise Exception("Model should be provided".capitalize())

    def l2(self, model):
        if model is not None:
            return (
                torch.norm(input=params, p=2) for params in model.parameters()
            ).sum()
        else:
            raise Exception("Model should be provided".capitalize())

    def elastic_net(self, model):
        if model is not None:
            l1 = self.l1(model=model)
            l2 = self.l2(model=model)

            return l1 + l2
        else:
            raise Exception("Model should be provided".capitalize())

    def save_checkpoints(self, **kwargs):
        if (
            (os.path.exists(TRAIN_MODELS))
            and (os.path.exists(BEST_MODELS))
            and os.path.exists(BEST_MODEL)
        ):
            torch.save(
                {
                    "netG": self.netG.state_dict(),
                    "netG_loss": kwargs["netG_loss"],
                },
                os.path.join(TRAIN_MODELS, "netG{}.pth".format(kwargs["epoch"])),
            )

            self.loss_track["netG"].append(kwargs["netG_loss"])
            self.loss_track["netD"].append(kwargs["netD_loss"])

            if self.infinity > kwargs["netG_loss"]:

                self.infinity = kwargs["netG_loss"]

                torch.save(
                    {
                        "netG": self.netG.state_dict(),
                        "netG_loss": kwargs["netG_loss"],
                    },
                    os.path.join(BEST_MODELS, "netG{}.pth".format(kwargs["epoch"])),
                )

        else:
            raise FileExistsError("The directory should be created".capitalize())

    def update_discriminator_training(self, **kwargs):
        try:
            self.optimizerD.zero_grad()

            hr_loss = self.adversarial_loss(
                self.netD(kwargs["hr_images"]), kwargs["real_labels"]
            )
            fake_loss = self.adversarial_loss(
                self.netD(self.netG(kwargs["lr_images"])), kwargs["fake_labels"]
            )

            total_loss = 0.5 * (hr_loss + fake_loss)

            if self.is_l2:
                total_loss += self.l1(self.netD)

            if self.is_elastic_net:
                total_loss += self.elastic_net(self.netD)

            total_loss.backward()

            if self.is_weight_clip:
                for params in self.netD.parameters():
                    params.data.clamp_(-self.clip_value, self.clip_value)

            self.optimizerD.step()

        except KeyError as e:
            print("The exception caught in (Discriminator) # {}".format(e).capitalize())

        except Exception as e:
            print("The exception caught in (Discriminator)# {}".format(e).capitalize())

        else:
            return total_loss.item()

    def update_generator_training(self, **kwargs):
        try:
            self.optimizerG.zero_grad()

            generated_hr = self.netG(kwargs["lr_images"])

            adversarial_loss = self.adversarial_loss(
                self.netD(generated_hr), kwargs["real_labels"]
            )

            real_features = self.criterion_content(kwargs["hr_images"])
            fake_features = self.criterion_content(generated_hr)

            content_loss_vgg = torch.abs(real_features - fake_features).mean()
            total_loss = self.content_loss * adversarial_loss + content_loss_vgg

            if self.is_l2:
                total_loss += self.l1(self.netG)

            if self.is_elastic_net:
                total_loss += self.elastic_net(self.netG)

            total_loss.backward()
            self.optimizerG.step()

        except KeyError as e:
            print("The Exception caught in (Generator) # {}".format(e).capitalize())

        except Exception as e:
            print("The Exception caught in (Generator) # {}".format(e).capitalize())

        else:
            return total_loss.item()

    def validate_model_on_test_data(self, **kwargs):
        try:
            generated_hr = self.netG(kwargs["lr_images"])

            loss = self.adversarial_loss(generated_hr, kwargs["hr_images"])

        except KeyError as e:
            print("The exception caught in # {}".format(e).capitalize())

        except Exception as e:
            print("The exception caught in # {}".format(e).capitalize())

        else:
            return loss.item()

    def save_training_images(self, **kwargs):
        lr_images, hr_images = next(iter(self.test_dataloader))
        lr_images = lr_images.to(self.device)
        hr_images = hr_images.to(self.device)

        generated_hr_images = self.netG(lr_images[0:8])

        if os.path.exists(TRAIN_IMAGES):
            save_image(
                generated_hr_images,
                os.path.join(TRAIN_IMAGES, "train_{}.png".format(kwargs["epoch"] + 1)),
                nrow=4,
                normalize=True,
            )
        else:
            raise FileExistsError("The directory should be created".capitalize())

    def show_progress(self, **kwargs):
        if self.is_display:
            print(
                "Epochs - [{}/{}] - train_netG_loss: {:.5f} - train_netD_loss: {:.5f} - test_loss: {:.5f}".format(
                    kwargs["epoch"],
                    kwargs["epochs"],
                    np.mean(kwargs["netG_loss"]),
                    np.mean(kwargs["netD_loss"]),
                    np.mean(kwargs["test_loss"]),
                )
            )
        else:
            print(
                "Epochs - [{}/{}] is completed".format(
                    kwargs["epoch"] + 1, kwargs["epochs"]
                )
            )

    def train(self):
        warnings.filterwarnings("ignore")

        for epoch in tqdm(range(self.epochs)):
            self.netG_loss = list()
            self.netD_loss = list()
            self.test_loss = list()

            for _, (lr_images, hr_images) in enumerate(self.train_dataloader):
                lr_images = lr_images.to(self.device)
                hr_images = hr_images.to(self.device)
                batch_size = hr_images.size(0)

                real_labels = torch.ones((batch_size,)).to(self.device)
                fake_labels = torch.zeros((batch_size,)).to(self.device)

                D_loss = self.update_discriminator_training(
                    lr_images=lr_images,
                    hr_images=hr_images,
                    real_labels=real_labels,
                    fake_labels=fake_labels,
                )
                G_loss = self.update_generator_training(
                    lr_images=lr_images, hr_images=hr_images, real_labels=real_labels
                )

                self.netG_loss.append(G_loss)
                self.netD_loss.append(D_loss)

            for _, (lr_images, hr_images) in enumerate(self.test_dataloader):
                lr_images = lr_images.to(self.device)
                hr_images = hr_images.to(self.device)

                loss = self.validate_model_on_test_data(
                    lr_images=lr_images, hr_images=hr_images
                )

                self.test_loss.append(loss)

            try:
                self.save_checkpoints(
                    epoch=epoch + 1,
                    netG_loss=np.array(self.netG_loss).mean(),
                    netD_loss=np.array(self.netD_loss).mean(),
                )
            except Exception as e:
                print("The exception caught in # {}".format(e).capitalize())

            else:
                self.save_training_images(epoch=epoch + 1)

                self.history["netG"].append(np.mean(self.netG_loss))
                self.history["netD"].append(np.mean(self.netD_loss))

            finally:
                self.show_progress(
                    epoch=epoch + 1,
                    epochs=self.epochs,
                    netG_loss=np.array(self.netG_loss).mean(),
                    netD_loss=np.array(self.netD_loss).mean(),
                    test_loss=np.array(self.test_loss).mean(),
                )

            if self.is_lr_scheduler:
                self.schedulerD.step()
                self.schedulerG.step()
        try:
            if os.path.exists(MODEL_HISTORY):
                pd.DataFrame(self.loss_track).to_csv(
                    os.path.join(MODEL_HISTORY, "history.csv")
                )
            else:
                raise FileExistsError("The directory should be created".capitalize())

        except Exception as e:
            print("The exception caught in # {}".format(e).capitalize())
        else:
            if os.path.exists(METRICS_PATH):
                dump(
                    value=self.history,
                    filename=os.path.join(METRICS_PATH, "history.pkl"),
                )
            else:
                raise FileExistsError(
                    "The directory should be created (Model History)".capitalize()
                )

    @staticmethod
    def plot_history():
        if os.path.exists(METRICS_PATH):
            history = load(filename=os.path.join(METRICS_PATH, "history.pkl"))

            plt.plot(history["netG"], label="netG_loss")
            plt.plot(history["netD"], label="netD_loss")
            plt.legend()
            plt.xlabel("Epochs".capitalize())
            plt.ylabel("Loss".capitalize())
            plt.show()

        else:
            raise FileExistsError(
                "The directory should be created (Model History)".capitalize()
            )

In [ ]:
if __name__ == "__main__":
    trainer = Trainer(epochs=5, lr=0.0002, device="mps", content_loss=1e-3)
    trainer.train()
    
    trainer.plot_history()

In [ ]:
import imageio

class Test:
    def __init__(self, device="mps", model=None):
        self.device = device_init(device=device)
        self.model = model
        self.infinity = float("inf")

        self.best_model = None
        self.netG = Generator().to(self.device)

    def load_dataset(self):
        if os.path.exists(PROCESSED_DATA_PATH):
            return load(
                filename=os.path.join(PROCESSED_DATA_PATH, "test_dataloader.pkl")
            )
        else:
            raise Exception("Dataset not found".capitalize())

    def select_best_model(self):
        if os.path.exists(BEST_MODELS):
            for model in os.listdir(BEST_MODELS):
                if model != ".DS_Store":
                    model_path = os.path.join(BEST_MODELS, model)
                    model_loss = torch.load(model_path)
                    model_loss = model_loss["netG_loss"]

                    if self.infinity > model_loss:
                        self.infinity = model_loss

                        self.best_model = torch.load(model_path)
                        self.best_model = self.best_model["netG"]

                return self.best_model
        else:
            raise Exception("Best models not found".capitalize())

    def create_gif(self):
        if (os.path.exists(GIF_FILE)) and (os.path.exists(TRAIN_IMAGES)):
            self.images = []

            for image in os.listdir(TRAIN_IMAGES):
                self.images.append(imageio.imread(os.path.join(TRAIN_IMAGES, image)))

            imageio.mimsave(os.path.join(GIF_FILE, "train_gif.gif"), self.images)

        else:
            raise Exception("GIF file not found".capitalize())

    def image_normalized(self, **kwargs):
        return (kwargs["image"] - kwargs["image"].min()) / (
            kwargs["image"].max() - kwargs["image"].min()
        )

    def plot_images(self, **kwargs):
        plt.figure(figsize=(80, 40))

        lr_images, hr_images = next(iter(self.load_dataset()))
        lr_images = lr_images.to(self.device)
        hr_images = hr_images.to(self.device)

        if self.model:
            self.netG.load_state_dict(torch.load(self.model)["netG"])
        else:
            self.netG.load_state_dict(self.select_best_model())

        generated_hr = self.netG(lr_images)

        for index, image in enumerate(generated_hr):
            lr = lr_images[index].permute(1, 2, 0).squeeze().cpu().detach().numpy()
            hr = hr_images[index].permute(1, 2, 0).squeeze().cpu().detach().numpy()
            gen_image = image.permute(1, 2, 0).squeeze().cpu().detach().numpy()

            lr = self.image_normalized(image=lr)
            hr = self.image_normalized(image=hr)
            gen_image = self.image_normalized(image=gen_image)

            plt.subplot(3 * 8, 3 * 8, 3 * index + 1)
            plt.imshow(lr, cmap="gray")
            plt.title("lr".lower())
            plt.axis("off")

            plt.subplot(3 * 8, 3 * 8, 3 * index + 2)
            plt.imshow(hr, cmap="gray")
            plt.title("hr".lower())
            plt.axis("off")

            plt.subplot(3 * 8, 3 * 8, 3 * index + 3)
            plt.imshow(gen_image, cmap="gray")
            plt.title("sr".upper())
            plt.axis("off")

        plt.tight_layout()

        if os.path.exists(TEST_IMAGES):
            plt.savefig(os.path.join(TEST_IMAGES, "result.png"))

        else:
            os.mkdir(TEST_IMAGES)
            raise Exception(
                "Could not create the image due to folder is not found".capitalize()
            )

        plt.show()

    def plot(self):
        try:
            self.plot_images(netG=self.netG)

        except Exception as e:
            print("The exception is: ", e)

        else:
            self.create_gif()


if __name__ == "__main__":
    chart = Test(device="mps")
    chart.plot()